In [1]:
from app.tools.db import  DatabaseConnection
from app.tools.constans import paramsl, paramsp
from app.tools.s3 import validate_path, client, bucket
from pprint import pprint as pp
import Levenshtein as lev
import ijson, json, os
import pandas as pd
import glob
import uuid
import re

path_input = "trash/dicts/old_matches.json"
conn = DatabaseConnection(connect= True, **paramsl)
conn1 = DatabaseConnection(connect= True, **paramsl)
conn2 = DatabaseConnection(connect= True, **paramsp)

def get_path(files: str, paths: str) -> str:
    for file in files:
        if file != None:
            path_files = [validate_path(os.path.join(path,file)) for path in paths]
            if any(path_files):
                return paths[path_files.index(True)]
    else:
        return None

In [3]:
paths_s3_ajio = "ajio-myntra/origin/20240131/",

routes = {
    "ajio-myntra/alternative/20240131/dev/": "trash/dicts/brands_dictionary_kids.json",
    "ajio-myntra/alternative/20240202/dev/": "trash/dicts/brands_dictionary_home.json",
    "ajio-myntra/alternative/20240131/dev/": "trash/dicts/brands_dictionary_men.json",
    "ajio-myntra/alternative/20240201/dev/": "trash/dicts/brands_dictionary_women.json"
}

paths_s3_myntra = {
    "ajio-myntra/alternative/20240131/dev/": [],
    "ajio-myntra/alternative/20240202/dev/": [],
    "ajio-myntra/alternative/20240201/dev/": []
}

brands_not_matched = pd.read_json("trash/dicts/not_matched.json")

for i, data in brands_not_matched.iterrows():

    ajio_brand = data["ajio_brand"]
    myntra_brand = data["myntra_brand"]

    query = "SELECT id, products FROM request_e07aaf8f_a587_42af_a0b3_0abd42a7ffc5.{table} where products ->> 'brand' = '{brand}' limit 1"

    conn1.execute(query.format(table = "alternative", brand = myntra_brand))

    result = conn1.result.fetchone()
    if result is not None:
        myntra_file_name = [name for name in result[1]["product_images"] if name is not None][0]
        # Resto del código para procesar myntra_file_name y realizar las operaciones necesarias
    else:
        # Manejar el caso cuando no se obtiene ninguna fila de la consulta
        print("No se encontraron filas en la consulta.")

    for path in paths_s3_myntra:

        s3_path = os.path.join(path, myntra_file_name)

        if validate_path(s3_path):
            paths_s3_myntra[path].append(data.to_dict())
            break


No se encontraron filas en la consulta.
No se encontraron filas en la consulta.


In [5]:
for s3_path, local_file in routes.items():
    brands_to_add = paths_s3_myntra[s3_path]

    with open(local_file, "r+") as file:
        data_dict = json.load(file)

        for brand in brands_to_add:
            data_dict.append(brand)

        file.seek(0)  # Mueve el puntero al inicio del archivo
        json.dump(data_dict, file, indent=4)
        file.truncate()  # Trunca el contenido restante si es más corto que el nuevo contenido


In [2]:
def extrac_one_fileName_from_s3(schema_name: str,  table_name: str, brand_name: str) -> str:

    conn = DatabaseConnection(connect=True, **paramsl)
    query = f"SELECT products FROM {schema_name}.{table_name} where products ->> 'brand' = %s limit 1"
    conn.execute(query, (brand_name, ))

    result = conn.result.fetchone()
    exist = result != None

    if exist:
        product_image = [file_name for file_name in result[0]["product_images"] if file_name != None][0]
    else:
        product_image = None
    
    return product_image

In [4]:
extrac_one_fileName_from_s3("request_e07aaf8f_a587_42af_a0b3_0abd42a7ffc5", "alternative", "FEETWELL SHOESSS")